In [33]:
import openai
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_AUTO_SIZE
from tqdm import tqdm
import os
import requests

In [34]:
filter = "ABCDEFGHIJKLMNOPQRSTUVWXYZ abcdefghijklmnopqrstuvwxyz!?,"
count = 0
structure = {}
lst = []

In [35]:
    
mainidea = str(input("Whats The Main Theme Of The Presentation?: ")) #e
numofsub = int(input("How many slides do you want ( Not including Opening and Ending )?: "))
length = str(input("Length Of The Text In The The Presentation (Medium to Short Recommended.): "))
undertitle = str(input("What will be written under the title?: (EX: By: Josh, Alex)?: "))


decision = str(input("From what do you want to create the presentation. \n (Informative Text [Text], A Single Subject [Subject], Define Each Slide [Define])?:  "))

if "define" in decision.lower().replace(" ", ""):
    titles = []
    for x in range(numofsub):
        titles.append(input(f"What do you want the title to be for informative slide {x + 1}: "))
    decision = "define"
    
elif "subject" in decision.lower().replace(" ", ""):
    print("Ok, Proceeding..")
    decision = "subject"
elif "text" in decision.lower().replace(" ", ""):
    print("Ok, Proceeding..")
    decision = "text"


Ok, Proceeding..


In [36]:
def cleanup(text):
    clean = ""
    for x in range(len(text)):
        if text[x] in filter:
            clean += text[x]
    return clean


In [37]:
def user_to_raw(user_title_list, length, mainidea, undertitle):
    raw = []
    raw.append({"title": mainidea, "content": undertitle})

    for x in tqdm(user_title_list):
        
        gpt = openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"create a {length} text length under 1300 charachters and under 17 lines (including the spaces as charachters) about {x} regarding to the subject {mainidea}: ",
                           temperature=0.3,
                           max_tokens=500,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]

        raw.append({"title": x.title(), "content": gpt})
    
    conc = openai.Completion.create(
                   model="text-davinci-003",
                   prompt=f"create a conclusion under 1300 charachters and under 17 lines (including the spaces as charachters) about {mainidea}: ",
                   temperature=0.3,
                   max_tokens=500,
                   top_p=1.0,
                   frequency_penalty=0.0,
                   presence_penalty=0.0)["choices"][0]["text"]

    raw.append({"title": "Conclusion", "content": conc})

    return raw    

In [38]:
def getsubjects(subject, parts):
    return (openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"generate {parts} diffrent titles which are questions for a presentation about {subject} to answer without numbers and splitted by ,",
                           temperature=0.3,
                           max_tokens=500,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]).split(",")


In [39]:
def get_slide(topic, length=None, type=None, optional=""):
    if type == "start":
        return {"title": cleanup(topic).title(),
                "content": optional}
    elif type == "middle":
        return {"title": cleanup(topic).title(),
                "content": openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"create a {length} text length under 1300 charachters and under 17 lines (including the spaces as charachters) about {topic}: ",
                           temperature=0.3,
                           max_tokens=640,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]}
    elif type == "end":
        return {"title": "Conclusion",
                "content": openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"create a conclusion for a presentation about the subject of {topic}: ",
                           temperature=0.3,
                           max_tokens=640,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]}
    else:
        return {"title": cleanup(topic).title(),
                "content": openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"create a {length} {type} length about {topic} under 1300 charachters and under 17 lines (including the spaces as charachters): ",
                           temperature=0.3,
                           max_tokens=640,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]}


In [40]:
def presentation_text(structure, title, undertitle):
    completed = [get_slide(title, optional= undertitle, type="start")]
    
    for x in structure:
        completed.append(get_slide(x, length=structure[x], type="middle"))
    
    completed.append(get_slide(title, type="end"))
    
    return completed

In [41]:
def FirstRun(mainidea, numofsub, length, undertitle, decision, text = ""):
    if "text" in decision.lower().replace(" ", ""):      
        lst = (openai.Completion.create(
                           model="text-davinci-003",
                           prompt=f"summarize a text to {numofsub} diffrent titles/questions that would fit a presentation regarding to this text {text}. the titles will be splitted by a ,",
                           temperature=0.3,
                           max_tokens=500,
                           top_p=1.0,
                           frequency_penalty=0.0,
                           presence_penalty=0.0)["choices"][0]["text"]).split(",")

        for x in range(numofsub):
            structure.update({lst[x]:length})
        raw = presentation_text(structure, mainidea, undertitle)
        return raw
 

    elif "subject" in decision.lower().replace(" ", ""):
        lst = getsubjects(mainidea, numofsub)
        for x in range(numofsub):
            structure.update({lst[x]:length})
        raw = presentation_text(structure, mainidea, undertitle)
        return raw

    else:
        print("Error, Try again")
        FirstRun(mainidea, numofsub, length, undertitle, decision)



In [42]:
def photomaker(mainidea):
    response = openai.Image.create(
      prompt=f"a basic photo, make it look realistic, suitable for a presentation, related to the subject: {mainidea}",
      n=4,
      size="1024x1024"
    )
    image_url = response['data'][0]['url']



    def download_image(url, file_name, headers):

        # Send GET request
        response = requests.get(url, headers=headers)

        # Save the image
        if response.status_code == 200:
            with open(file_name, "wb") as f:
                f.write(response.content)
        else:
            print(response.status_code)

    download_image(image_url, f"Picture for presentation.png",     headers = {
            "User-Agent": "Chrome/51.0.2704.103",
        }
    )


In [43]:
def presentation_maker(raw, numofsub):
    global counter
    global count
    typelist = ["start", "ending"]
    for x in range(numofsub):
        typelist.insert(1, "middle")
    def prsmaker(type, text="", maintitle=""):
        global counter
        prs.slide_width = Inches(16)
        prs.slide_height = Inches(9)
        if type == "start":
            lyt = prs.slide_layouts[2]  # choosing a slide layout
            slide = prs.slides.add_slide(lyt)  # adding a slide
            title = slide.shapes.title  # assigning a title
            left = Inches(7)
            top = Inches(1)
            pic = slide.shapes.add_picture("Picture for presentation.png", left, top)
            height = Inches(7)
            width = Inches(7)
            pic.height = height
            pic.width = width
            subtitle = slide.placeholders[1]  # placeholder for subtitle
            title.text = maintitle  # title
            left = Inches(7)
            top = Inches(1)
            height = Inches(7)
            width = Inches(7)
            if text != "":
                subtitle.text = text  # subtitle
                subtitle.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
                subtitle.text_frame.word_wrap = False

        elif type == "middle":
            lyt = prs.slide_layouts[1]  # choosing a slide layout
            slide = prs.slides.add_slide(lyt)  # adding a slide
            title = slide.shapes.title  # assigning a title
            title.text = maintitle
            textBox = slide.placeholders[1]  # placeholder for subtitle
            textFrame = textBox.text_frame
            textFrame.word_wrap = True
            textParagraph = textFrame.add_paragraph()
            textParagraph.text = text
            textParagraph.autosize = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
            textFrame.fit_text(font_family='Calibri', max_size=28, bold=False, italic=False, font_file=None)

            textBox.text_frame.word_wrap = True
            textBox.width = Inches(15)
            textBox.height = Inches(9)  # text box height = 80% of slide height
            title.width = Inches(14)
            title.height = Inches(1)  # text box height = 80% of slide height

        elif type == "ending":
            lyt = prs.slide_layouts[1]  # choosing a slide layout
            slide = prs.slides.add_slide(lyt)  # adding a slide
            title = slide.shapes.title  # assigning a title
            title.text = "Conclusion"
            subtitle = slide.placeholders[1]  # placeholder for subtitle
            subtitle.text = text
            subtitle.text_frame.auto_size = MSO_AUTO_SIZE.TEXT_TO_FIT_SHAPE
            subtitle.text_frame.word_wrap = False
            subtitle.width = Inches(15)
            subtitle.height = Inches(9)  # text box height = 80% of slide height
            title.width = Inches(14)
            title.height = Inches(1)  # text box height = 80% of slide height

        if counter + 1 == len(typelist):
            prs.save(mainidea + " (Auto Slides) .pptx")  # saving file


    prs = Presentation()


    for x in tqdm(raw):
        prsmaker(typelist[count], text=x["content"], maintitle=x["title"])
        counter += 1
        count += 1

In [44]:
if decision == "define":
    photomaker(mainidea)
    counter = 0
    raw = user_to_raw(titles, length, mainidea, undertitle)
    presentation_maker(raw, numofsub)
elif decision == "subject":
    photomaker(mainidea)
    counter = 0
    raw = FirstRun(mainidea, numofsub, length, undertitle, decision)
    presentation_maker(raw, numofsub)
elif decision == "text":
    text = str(input("Paste the text down below:\n "))
    photomaker(mainidea)
    counter = 0
    raw = FirstRun(mainidea, numofsub, length, undertitle, decision, text)
    presentation_maker(raw, numofsub)

os.remove("Picture for presentation.png")


100%|██████████| 4/4 [00:02<00:00,  1.39it/s]


In [45]:
raw

[{'title': 'Cats', 'content': 'by: aviv'},
 {'title': 'Exploring The Fascinating World Of Cats',
  'content': '\n\nCats are one of the most beloved animals on the planet. They bring us joy, companionship, and unconditional love. Exploring the fascinating world of cats can be an exciting and rewarding experience. From their unique personalities to their playful antics, cats have something to offer everyone. \n\nCats come in all shapes and sizes, from the majestic Maine Coon to the tiny teacup Persian. Each breed has its own distinct characteristics, from their fur color and texture to their temperament. With so many breeds to choose from, there is sure to be the perfect cat for everyone. \n\nCats are also known for their intelligence and problem-solving skills. They can be taught tricks, play games, and even use the litter box. Cats also have a keen sense of hearing and smell, making them great hunters. \n\nCats are also great companions. They can provide comfort and companionship in ti